<a href="https://colab.research.google.com/github/johnnykindermann/forest_fire_detection/blob/main/wildfire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00


In [2]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import HTML, display
from base64 import b64encode

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def resize_frame(frame, max_width=1280):
    """
    Resizes a frame to a maximum width while maintaining aspect ratio.
    """
    h, w = frame.shape[:2]
    if w > max_width:
        ratio = max_width / w
        new_h = int(h * ratio)
        return cv2.resize(frame, (max_width, new_h), interpolation=cv2.INTER_AREA)
    return frame

def draw_detections(frame, results, model):
    """
    Draws bounding boxes, class names, and confidence scores on the frame.
    """
    # Define a color for the "fire" class (B, G, R)
    fire_color = (0, 0, 255) # Red

    # Ensure frame is writeable by making a copy
    frame_with_detections = frame.copy()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].int().tolist()
            conf = box.conf[0].item()
            cls_id = box.cls[0].int().item()
            cls_name = model.names[cls_id]

            # 1. Draw the bounding box
            cv2.rectangle(frame_with_detections, (x1, y1), (x2, y2), fire_color, 2)

            # 2. Create and draw the label
            label = f'{cls_name}: {conf:.2f}'
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame_with_detections, (x1, y1 - h - 10), (x1 + w, y1 - 5), fire_color, -1)
            cv2.putText(frame_with_detections, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return frame_with_detections

In [4]:
SOURCE_PATH='./wildfire_video.mp4'

In [5]:
try:
    model = YOLO('./wildfire-detection.pt')
except NameError:
    print("MODEL_PATH is not set. Please run Cell 4 to upload your model.")
except Exception as e:
    print(f"Error loading model: {e}")

try:
    file_ext = os.path.splitext(SOURCE_PATH)[1].lower()
    image_ext = ['.jpg', '.jpeg', '.png', '.bmp', '.webp']
    video_ext = ['.mp4', '.avi', '.mov', '.mkv']

    is_video = False
    output_path = 'output_demo.mp4'

    if file_ext in image_ext:
        print(f"Processing image: {SOURCE_PATH}")
        frame = cv2.imread(SOURCE_PATH)

        if frame is None:
            print(f"Error: Unable to read image file {SOURCE_PATH}")
        else:
            original_frame = frame.copy()
            results = model(frame)
            inferenced_frame = draw_detections(frame, results, model)

            combined_frame = cv2.hconcat([original_frame, inferenced_frame])
            frame_resized = resize_frame(combined_frame, max_width=1280)

            print("Displaying side-by-side result (Original vs. AI):")
            cv2_imshow(frame_resized)

    elif file_ext in video_ext:
        is_video = True
        print(f"Processing video: {SOURCE_PATH}. This may take a moment...")
        cap = cv2.VideoCapture(SOURCE_PATH)

        if not cap.isOpened():
            print(f"Error: Unable to open video file {SOURCE_PATH}")
        else:
            writer = None
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                original_frame = frame.copy()
                results = model(frame, stream=True)
                # Pass the original 'frame' to be drawn on
                inferenced_frame = draw_detections(frame, results, model)

                combined_frame = cv2.hconcat([original_frame, inferenced_frame])
                # Resize for display
                frame_resized = resize_frame(combined_frame, max_width=1280)

                # Initialize writer on first frame
                if writer is None:
                    h, w = frame_resized.shape[:2]
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    fps = int(cap.get(cv2.CAP_PROP_FPS))
                    writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

                writer.write(frame_resized)

            if writer:
                writer.release()
            cap.release()
            print(f"✅ Video processing finished. Output saved to {output_path}")
            print("Run the next cell to display the video.")

    else:
        print(f"Error: Unsupported file format {file_ext}")

except NameError:
    print("SOURCE_PATH is not set. Please run Cell 5 to upload your source file.")
except Exception as e:
    print(f"An error occurred: {e}")

Processing video: ./wildfire_video.mp4. This may take a moment...

0: 352x640 (no detections), 75.4ms
Speed: 16.5ms preprocess, 75.4ms inference, 0.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 7.5ms
Speed: 3.4ms preprocess, 7.5ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 6.8ms
Speed: 2.2ms preprocess, 6.8ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 6.6ms
Speed: 2.6ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 7.0ms
Speed: 2.9ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 6.0ms
Speed: 1.9ms preprocess, 6.0ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 10.8ms
Speed: 2.4ms preprocess, 10.8ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 